<a href="https://colab.research.google.com/github/chauhanneha16/Invoice/blob/main/invoicecorrect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install easyocr
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 20.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [26]:
import easyocr
import pandas as pd
import glob

img = glob.glob('/content/New_Project__9_.jpg')

for i, image in enumerate(img):
    reader = easyocr.Reader(['en'], gpu=True)
    results = reader.readtext(
        image,
        # Paragraph is needed to get text grouped together
        paragraph=True,
        # X_threshold is needed to get text grouped together
        # For example, address fields with high X_threshold will be grouped together
        # and will be returned as one text. This is not the desired outcome.
        # Lowering the X_threshold will return the address fields as separate text, which is also bad!
        # PS. Play around with the X_threshold to see the results.
        x_ths=0.9
    )
    df = pd.DataFrame(results, columns=['bbox', 'text'])
    print(df.head())

                                                bbox  \
0  [[1216, 199], [2320, 199], [2320, 438], [1216,...   
1   [[175, 733], [299, 733], [299, 778], [175, 778]]   
2  [[351, 729], [1076, 729], [1076, 913], [351, 9...   
3  [[1744, 743], [1922, 743], [1922, 795], [1744,...   
4  [[2140, 747], [2317, 747], [2317, 791], [2140,...   

                                                text  
0  The Car Doctors Ltd. 547 High Street, London C...  
1                                           Bill to:  
2  John Smith 123 Piccadilly, London +44781234567...  
3                                           Invoice:  
4                                           2021-009  


In [29]:
import easyocr
import pandas as pd
import glob
import re

data_output = [
    [
        'Invoice',
        'Country',
        'Invoice Date',  # Added 'Invoice Date' to the output headers
        'Subtotal',
        'VAT ID number',
        'VAT %',
        'VAT Amount',
        'Total',
    ]
]

img = glob.glob('/content/New_Project__9_.jpg')

for i, image in enumerate(img):
    reader = easyocr.Reader(['en'], gpu=True)
    results = reader.readtext(
        image,
        paragraph=True,
        x_ths=0.3
    )
    df = pd.DataFrame(results, columns=['bbox', 'text'])
    df.drop(columns=['bbox'], inplace=True)

    text = '\n\n'.join(df['text'].tolist())

    invoice = re.search(r'Invoice:\s*(.+?)\s*\n', text)
    invoice = invoice.group(1).strip() if invoice else None

    country_match = re.search(r'Address:\s*(.+?)\s*\n\n', text, re.DOTALL)
    country = country_match.group(1).split(',')[-1].strip() if country_match else None

    invoice_date_match = re.search(r'Date:\s*(.+?)\s*\n', text)  # Regular expression to extract invoice date
    invoice_date = invoice_date_match.group(1).strip() if invoice_date_match else None  # Extract invoice date if found

    subtotal_amount_match = re.search(r'Subtotal \s*(.+?)\s*\n', text)
    subtotal_amount = subtotal_amount_match.group(1).strip() if subtotal_amount_match else None

    vat_id_number_match = re.search(r'VAT number:\s*(.+?)\s*\n', text)
    vat_id_number = vat_id_number_match.group(1).strip() if vat_id_number_match and vat_id_number_match.group(1) != '999 3333 11' else None

    vat_information = re.search(r'VAT \((.+?[%9])\):\s*(.+?)\s*\n', text)
    if not vat_information:
        print('VAT information not found for invoice: ' + image)
        vat_percent = None
        vat_amount = None
    else:
        vat_percent = vat_information.group(1).strip()
        vat_amount = vat_information.group(2).strip()

    total_amount_match = re.search(r'Invoice Total \(USD\):\s*(.+?)\s*\n', text)
    total_amount = total_amount_match.group(1).strip() if total_amount_match else None

    data_output.append([
        invoice,
        country,
        invoice_date,  # Added invoice_date to the data output
        subtotal_amount,
        vat_id_number,
        vat_percent,
        vat_amount,
        total_amount,
    ])

print(data_output)



VAT information not found for invoice: /content/New_Project__9_.jpg
[['Invoice', 'Country', 'Invoice Date', 'Subtotal', 'VAT ID number', 'VAT %', 'VAT Amount', 'Total'], ['2021-009', None, '19/05/2021', 'without VAT', None, None, None, None]]


In [15]:
import easyocr
import pandas as pd
import glob
import re

data_output = [
    [
        'Invoice number',
        'Country',
        'Subtotal',
        'VAT ID number',
        'VAT %',
        'VAT Amount',
        'Total',
    ]
]

img = glob.glob('/content/invoice.jpg')

for i, image in enumerate(img):
    reader = easyocr.Reader(['en'], gpu=True)
    results = reader.readtext(
        image,
        # Paragraph is needed to get text grouped together
        paragraph=True,
        # X_threshold is needed to get text grouped together
        # For example, address fields with high X_threshold will be grouped together
        # and will be returned as one text. This is not the desired outcome.
        # Lowering the X_threshold will return the address fields as separate text, which is also bad!
        # PS. Play around with the X_threshold to see the results.
        x_ths=0.3
    )
    df = pd.DataFrame(results, columns=['bbox', 'text'])
    df.drop(columns=['bbox'], inplace=True)

    text = '\n\n'.join(df['text'].tolist())

    invoice_number_match = re.search(r'No.\s*(.+?)\s*\n', text)
    invoice_number = invoice_number_match